In [1]:
using Plots
using CSV
using ControlSystems

gr()

Plots.GRBackend()

In [2]:
#体を揺すって脚を出す歩容
#確定版2019/01/20

zc=0.11 #0.12
g=9.8
dt=0.0025
L=0.15 #0.16
l=0.025 #歩幅
#x=0.03
ft=0.05 #最初の全脚支持期間

#support time
Tsup=Int32(0.4/dt)
#both foot support point time
Tdl=Int32(0.0025/dt) 
#preview step
M=600 #150

rotate_r=
            #r             c         l
real_sx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L-l; #第一ZMP
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L-l;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L-l; #一度動かない瞬間を作る
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L-l;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L-l;
        (1/2)*sqrt(3)*L; 0.0-l; (1/2)*sqrt(3)*L-l;#第二zmp
        (1/2)*sqrt(3)*L; 0.0-l; (1/2)*sqrt(3)*L-l;
        (1/2)*sqrt(3)*L; 0.0-l; (1/2)*sqrt(3)*L-l;
        (1/2)*sqrt(3)*L; 0.0-l; (1/2)*sqrt(3)*L-l;
         
        (1/2)*sqrt(3)*L; 0.0-l; (1/2)*sqrt(3)*L-l;#stop time
    
        (1/2)*sqrt(3)*L-l; 0.0-l; (1/2)*sqrt(3)*L-l;#第3zmp
    
        (1/2)*sqrt(3)*L-l; 0.0-l; (1/2)*sqrt(3)*L-l;
        (1/2)*sqrt(3)*L-l; 0.0-l; (1/2)*sqrt(3)*L-l;
        
        ] #  rcl

real_sy=[0.0; L/2; L;
    
        0.0; L/2; L;
         0.0;  L/2; L;
    
        0.0;  L/2; L; #動かない瞬間を作る
        
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;
        
        0.0;  L/2; L; #sptime
        0.0;  L/2; L;
        
        0.0;  L/2; L;#動かない期間
    
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;

        0.0;  L/2; L;
         ]

pltsx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L] #plot用
pltsy=[0.0; L/2; L]

leg_sign=[0;3;0;0;0;0;2;0;0;0;0;1;0;0;]#上げる脚とステップの関係 #left:3 right:1 center:

#初期姿勢
f_zmp_x=abs(real_sx[1]-real_sx[2])/2
f_zmp_y=L/2+0.00

e_zmp_x=abs(real_sx[1]-real_sx[2])/2-l
e_zmp_y=L/2+0.00 #追加

#Virtual support point
sx=[f_zmp_x
    real_sx[8]+abs(real_sx[4]-real_sx[8])/2+0.005+0.00
    abs(real_sx[1]-real_sx[2])/2-(l/3)
    
    abs(real_sx[1]-real_sx[2])/2-(l/3) #動かない瞬間をつくる
    
    abs(real_sx[1]-real_sx[2])/2-(l/3)+0.05 #sp time
    abs(real_sx[1]-real_sx[2])/2-(l/3)+0.05 #stop time
    real_sx[1]-(l/2)+0.00 #支持脚線の第二ZMP
    abs(real_sx[1]-real_sx[2])/2-(l/3)*2#+0.06
    
    abs(real_sx[1]-real_sx[2])/2-(l/3)*2#+0.05 #stop time
    abs(real_sx[1]-real_sx[2])/2-(l/3)*2 #sp time
    abs(real_sx[1]-real_sx[2])/2-(l/3)*2 #stop time
    
    real_sx[1]/2-l+0.00 #第三zmp 0.005
    abs(real_sx[1]-real_sx[2])/2-l+0.012 
    
    e_zmp_x]

sy=[f_zmp_y
    abs(real_sy[7]-real_sy[8])/2+0.005+0.003
    L/2
    
    L/2 #一度何もしない瞬間を作る
    
    L/2 #sp time
    L/2#stoptime
    L/2 #ZMP2
    L/2
    
    L/2#stop time
    L/2#sptime
    L/2#stoptime
    
    real_sy[2]+abs(real_sy[2]-real_sy[3])/2-0.004 #第三zmp
    L/2
    
    e_zmp_y]

vpoint_sum=length(sx)
full_leg_step_sum=vpoint_sum-1

#to generate trajectory of ZMP when both foot support point 
x_tdl_a=zeros(Float64,full_leg_step_sum)
x_tdl_b=zeros(Float64,full_leg_step_sum)
y_tdl_a=zeros(Float64,full_leg_step_sum)
y_tdl_b=zeros(Float64,full_leg_step_sum)

for i in 1:full_leg_step_sum
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:full_leg_step_sum
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#target ZMP vector
px_ref=zeros(vpoint_sum*Tsup+M+1 + full_leg_step_sum*Tdl)
py_ref=zeros(px_ref)

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            px_ref[k+step]=sx[i]
        end
    else
        for k in 1:Tsup
            px_ref[k+step]=sx[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            px_ref[s+step]=x_tdl_a[i]*(s+step)*dt+x_tdl_b[i]
        end
        step=step+Tdl
    end
end

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            py_ref[k+step]=sy[i]
        end
    else
        for k in 1:Tsup
            py_ref[k+step]=sy[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            py_ref[s+step]=y_tdl_a[i]*(s+step)*dt+y_tdl_b[i]
        end
        step=step+Tdl
    end
end

#total step
k_max=length(px_ref)-M
#computed ZMP vector
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max);

In [3]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[20000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#state vector
x=[ sx[1]
    0.0
    0.0]

y=[ sy[1]
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0];

In [4]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #to record for plot
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #to record for plot
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 

xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y

dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR
old_x=x
old_y=y
xu=0
yu=0
P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 0.0

In [5]:
for i in 1:k_max-1
    
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]    
    k+=1
    x_plot_log[k+1]=x[1] #to record for plot
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #to record for plot
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    dx=x-old_x
    dy=y-old_y
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    old_x=x
    old_y=y
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end

In [6]:
plot(t*dt,px_ref,lab="Target ZMP",linewidth=3,linecolor="black")
plot!(t*dt,x_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t*dt,px,lab="Target ZMP",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP_x[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 time[ms] 
 
 
 ZMP 
 
 
 x[m] 
 
<polyline clip-path="url(#clip4802)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,173.45 39.6154,173.45 39.8608,173.45 40.1061,173.45 40.3514,173.45 40.5968,173.45 40.8421,173.45 41.0875,173.45 41.3328,173.45 41.5782,173.45 
 41.8235,173.45 42.0688,173.45 42.3142,173.45 42.5595,173.45 42.8049,173.45 43.0502,173.45 43.2956,173.45 43.5409,173.45 43.7862,173.45 44.0316,173.45 
 44.2769,173.45 44.5223,173.45 44.7676,173.45 45.0129,173.45 45.2583,173.45 45.5036,173.45 45.749,173.45 45.9943,173.45 46.2397,173.45 46.485,173.45 
 46.7303,173.45 46.9757,173.45 47.221,173.45 47.4664,173.45 47.7117,173.45 47.9571,173.45 48.2024,173.45 48.4477,173.45 48.6931,173.45 48.9384,173.45 
 49.1838,173.45 49.4291,173.45 49.6744,173.45 49.9198,173.45 50.1651,173.45 50.4105,173.45 50.6558,173.45 50.9012,173.45 51.1465,173.45 51.3918,173.45 
 51.6372,173.45 51.8825,173.45 52.1279,173.45 52.3732,173.45 52.6186,173.45 52.8639,173.45 53.1092,173.45 53.3546,173.45 53.5999,173.45 53.8453,173.45 
 54.0906,173.45 54.336,173.45 54.5813,173.45 54.8266,173.45 55.072,173.45 55.3173,173.45 55.5627,173.45 55.808,173.45 56.0533,173.45 56.2987,173.45 
 56.544,173.45 56.7894,173.45 57.0347,173.45 57.2801,173.45 57.5254,173.45 57.7707,173.45 58.0161,173.45 58.2614,173.45 58.5068,173.45 58.7521,173.45 
 58.9975,173.45 59.2428,173.45 59.4881,173.45 59.7335,173.45 59.9788,173.45 60.2242,173.45 60.4695,173.45 60.7148,173.45 60.9602,173.45 61.2055,173.45 
 61.4509,173.45 61.6962,173.45 61.9416,173.45 62.1869,173.45 62.4322,173.45 62.6776,173.45 62.9229,173.45 63.1683,173.45 63.4136,173.45 63.659,173.45 
 63.9043,173.45 64.1496,173.45 64.395,173.45 64.6403,173.45 64.8857,173.45 65.131,173.45 65.3763,173.45 65.6217,173.45 65.867,173.45 66.1124,173.45 
 66.3577,173.45 66.6031,173.45 66.8484,173.45 67.0937,173.45 67.3391,173.45 67.5844,173.45 67.8298,173.45 68.0751,173.45 68.3205,173.45 68.5658,173.45 
 68.8111,173.45 69.0565,173.45 69.3018,173.45 69.5472,173.45 69.7925,173.45 70.0378,173.45 70.2832,173.45 70.5285,173.45 70.7739,173.45 71.0192,173.45 
 71.2646,173.45 71.5099,173.45 71.7552,173.45 72.0006,173.45 72.2459,173.45 72.4913,173.45 72.7366,173.45 72.982,173.45 73.2273,173.45 73.4726,173.45 
 73.718,173.45 73.9633,173.45 74.2087,173.45 74.454,173.45 74.6994,173.45 74.9447,173.45 75.19,173.45 75.4354,173.45 75.6807,173.45 75.9261,173.45 
 76.1714,173.45 76.4167,173.45 76.6621,173.45 76.9074,173.45 77.1528,173.45 77.3981,173.45 77.6435,173.45 77.8888,173.45 78.1341,173.45 78.3795,173.45 
 78.6248,158.434 78.8702,158.434 79.1155,158.434 79.3609,158.434 79.6062,158.434 79.8515,158.434 80.0969,158.434 80.3422,158.434 80.5876,158.434 80.8329,158.434 
 81.0782,158.434 81.3236,158.434 81.5689,158.434 81.8143,158.434 82.0596,158.434 82.305,158.434 82.5503,158.434 82.7956,158.434 83.041,158.434 83.2863,158.434 
 83.5317,158.434 83.777,158.434 84.0224,158.434 84.2677,158.434 84.513,158.434 84.7584,158.434 85.0037,158.434 85.2491,158.434 85.4944,158.434 85.7397,158.434 
 85.9851,158.434 86.2304,158.434 86.4758,158.434 86.7211,158.434 86.9665,158.434 87.2118,158.434 87.4571,158.434 87.7025,158.434 87.9478,158.434 88.1932,158.434 
 88.4385,158.434 88.6839,158.434 88.9292,158.434 89.1745,158.434 89.4199,158.434 89.6652,158.434 89.9106,158.434 90.1559,158.434 90.4012,158.434 90.6466,158.434 
 90.8919,158.434 91.1373,158.434 91.3826,158.434 91.628,158.434 91.8733,158.434 92.1186,158.434 92.364,158.434 92.6093,158.434 92.8547,158.434 93.1,158.434 
 93.3454,158.434 93.5907,158.434 93.836,158.434 94.0814,158.434 94.3267,158.434 94.5721,158.434 94.8174,158.434 95.0628,158.434 95.3081,158.434 95.5534,158.434 
 95.7988,158.434 96.0441,158.434 96.2895,158.434 96.5348,158.434 96.7801,158.434 97.0255,15

In [7]:
plot(t,py_ref,lab="Target ZMP y",linewidth=3,linecolor="black")
plot!(t,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t,py,lab="ZMP y",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 300 
 
 
 600 
 
 
 900 
 
 
 1200 
 
 
 1500 
 
 
 1800 
 
 
 2100 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 time[ms] 
 
 
 ZMP[m] 
 
<polyline clip-path="url(#clip4902)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,123.098 39.6154,123.098 39.8608,123.098 40.1061,123.098 40.3514,123.098 40.5968,123.098 40.8421,123.098 41.0875,123.098 41.3328,123.098 41.5782,123.098 
 41.8235,123.098 42.0688,123.098 42.3142,123.098 42.5595,123.098 42.8049,123.098 43.0502,123.098 43.2956,123.098 43.5409,123.098 43.7862,123.098 44.0316,123.098 
 44.2769,123.098 44.5223,123.098 44.7676,123.098 45.0129,123.098 45.2583,123.098 45.5036,123.098 45.749,123.098 45.9943,123.098 46.2397,123.098 46.485,123.098 
 46.7303,123.098 46.9757,123.098 47.221,123.098 47.4664,123.098 47.7117,123.098 47.9571,123.098 48.2024,123.098 48.4477,123.098 48.6931,123.098 48.9384,123.098 
 49.1838,123.098 49.4291,123.098 49.6744,123.098 49.9198,123.098 50.1651,123.098 50.4105,123.098 50.6558,123.098 50.9012,123.098 51.1465,123.098 51.3918,123.098 
 51.6372,123.098 51.8825,123.098 52.1279,123.098 52.3732,123.098 52.6186,123.098 52.8639,123.098 53.1092,123.098 53.3546,123.098 53.5999,123.098 53.8453,123.098 
 54.0906,123.098 54.336,123.098 54.5813,123.098 54.8266,123.098 55.072,123.098 55.3173,123.098 55.5627,123.098 55.808,123.098 56.0533,123.098 56.2987,123.098 
 56.544,123.098 56.7894,123.098 57.0347,123.098 57.2801,123.098 57.5254,123.098 57.7707,123.098 58.0161,123.098 58.2614,123.098 58.5068,123.098 58.7521,123.098 
 58.9975,123.098 59.2428,123.098 59.4881,123.098 59.7335,123.098 59.9788,123.098 60.2242,123.098 60.4695,123.098 60.7148,123.098 60.9602,123.098 61.2055,123.098 
 61.4509,123.098 61.6962,123.098 61.9416,123.098 62.1869,123.098 62.4322,123.098 62.6776,123.098 62.9229,123.098 63.1683,123.098 63.4136,123.098 63.659,123.098 
 63.9043,123.098 64.1496,123.098 64.395,123.098 64.6403,123.098 64.8857,123.098 65.131,123.098 65.3763,123.098 65.6217,123.098 65.867,123.098 66.1124,123.098 
 66.3577,123.098 66.6031,123.098 66.8484,123.098 67.0937,123.098 67.3391,123.098 67.5844,123.098 67.8298,123.098 68.0751,123.098 68.3205,123.098 68.5658,123.098 
 68.8111,123.098 69.0565,123.098 69.3018,123.098 69.5472,123.098 69.7925,123.098 70.0378,123.098 70.2832,123.098 70.5285,123.098 70.7739,123.098 71.0192,123.098 
 71.2646,123.098 71.5099,123.098 71.7552,123.098 72.0006,123.098 72.2459,123.098 72.4913,123.098 72.7366,123.098 72.982,123.098 73.2273,123.098 73.4726,123.098 
 73.718,123.098 73.9633,123.098 74.2087,123.098 74.454,123.098 74.6994,123.098 74.9447,123.098 75.19,123.098 75.4354,123.098 75.6807,123.098 75.9261,123.098 
 76.1714,123.098 76.4167,123.098 76.6621,123.098 76.9074,123.098 77.1528,123.098 77.3981,123.098 77.6435,123.098 77.8888,123.098 78.1341,123.098 78.3795,123.098 
 78.6248,219.625 78.8702,219.625 79.1155,219.625 79.3609,219.625 79.6062,219.625 79.8515,219.625 80.0969,219.625 80.3422,219.625 80.5876,219.625 80.8329,219.625 
 81.0782,219.625 81.3236,219.625 81.5689,219.625 81.8143,219.625 82.0596,219.625 82.305,219.625 82.5503,219.625 82.7956,219.625 83.041,219.625 83.2863,219.625 
 83.5317,219.625 83.777,219.625 84.0224,219.625 84.2677,219.625 84.513,219.625 84.7584,219.625 85.0037,219.625 85.2491,219.625 85.4944,219.625 85.7397,219.625 
 85.9851,219.625 86.2304,219.625 86.4758,219.625 86.7211,219.625 86.9665,219.625 87.2118,219.625 87.4571,219.625 87.7025,219.625 87.9478,219.625 88.1932,219.625 
 88.4385,219.625 88.6839,219.625 88.9292,219.625 89.1745,219.625 89.4199,219.625 89.6652,219.625 89.9106,219.625 90.1559,219.625 90.4012,219.625 90.6466,219.625 
 90.8919,219.625 91.1373,219.625 91.3826,219.625 91.628,219.625 91.8733,219.625 92.1186,219.625 92.364,219.625 92.6093,219.625 92.8547,219.625 93.1,219.625 
 93.3454,219.625 93.5907,219.625 93.836,219.625 94.0814,21

In [8]:
plot(x_plot_log,y_plot_log,lab="CoM",linewidth=1,linecolor="blue")
plot!(px,py,lab="ZoM",linewidth=1,linecolor="black") #追加
plot!(real_sx,real_sy,st=:scatter)
plot!(sx,sy,st=:scatter)
a=[0.15 0.15]
b=[0.19 0.19]
plot!(b,a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.03 
 
 
 0.00 
 
 
 0.03 
 
 
 0.06 
 
 
 0.09 
 
 
 0.12 
 
 
 0.15 
 
 
 0.18 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
<polyline clip-path="url(#clip5002)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 273.186,188.189 273.186,188.189 273.186,188.189 273.186,188.189 273.186,188.189 273.186,188.189 273.186,188.189 273.186,188.19 273.187,188.19 273.187,188.191 
 273.187,188.192 273.187,188.194 273.187,188.196 273.188,188.198 273.188,188.201 273.189,188.205 273.19,188.209 273.191,188.214 273.192,188.22 273.193,188.226 
 273.194,188.234 273.196,188.242 273.197,188.252 273.199,188.262 273.201,188.274 273.203,188.287 273.206,188.301 273.208,188.316 273.211,188.333 273.214,188.351 
 273.218,188.37 273.221,188.391 273.225,188.413 273.229,188.437 273.234,188.462 273.238,188.489 273.243,188.518 273.249,188.548 273.254,188.579 273.26,188.612 
 273.266,188.647 273.272,188.684 273.279,188.722 273.286,188.762 273.293,188.804 273.301,188.847 273.309,188.893 273.317,188.94 273.325,188.989 273.334,189.039 
 273.343,189.092 273.353,189.146 273.362,189.203 273.372,189.261 273.383,189.321 273.394,189.383 273.405,189.447 273.416,189.513 273.428,189.581 273.44,189.65 
 273.453,189.722 273.465,189.796 273.479,189.873 273.492,189.951 273.506,190.031 273.52,190.114 273.535,190.198 273.55,190.286 273.566,190.375 273.582,190.467 
 273.598,190.561 273.615,190.657 273.632,190.756 273.65,190.858 273.668,190.962 273.686,191.068 273.705,191.178 273.725,191.29 273.745,191.405 273.765,191.523 
 273.786,191.644 273.808,191.768 273.83,191.896 273.853,192.026 273.876,192.16 273.9,192.297 273.924,192.438 273.949,192.583 273.975,192.731 274.001,192.883 
 274.029,193.038 274.056,193.198 274.085,193.363 274.114,193.531 274.144,193.704 274.175,193.881 274.207,194.063 274.239,194.25 274.273,194.441 274.307,194.638 
 274.342,194.84 274.379,195.047 274.416,195.26 274.454,195.479 274.493,195.703 274.533,195.933 274.574,196.169 274.616,196.411 274.66,196.66 274.704,196.915 
 274.75,197.177 274.797,197.445 274.845,197.721 274.894,198.003 274.945,198.293 274.997,198.59 275.05,198.894 275.105,199.206 275.16,199.526 275.218,199.853 
 275.276,200.189 275.336,200.532 275.398,200.884 275.461,201.244 275.525,201.612 275.591,201.988 275.658,202.373 275.727,202.766 275.797,203.167 275.868,203.577 
 275.942,203.996 276.016,204.423 276.092,204.858 276.17,205.302 276.249,205.754 276.329,206.215 276.411,206.683 276.494,207.16 276.579,207.645 276.665,208.137 
 276.752,208.638 276.84,209.145 276.93,209.66 277.021,210.182 277.113,210.711 277.206,211.247 277.301,211.788 277.396,212.336 277.492,212.889 277.589,213.447 
 277.687,214.011 277.785,214.579 277.885,215.151 277.984,215.726 278.084,216.305 278.185,216.887 278.286,217.471 278.387,218.057 278.488,218.644 278.59,219.232 
 278.691,219.821 278.792,220.409 278.893,220.997 278.994,221.584 279.094,222.169 279.194,222.753 279.293,223.335 279.392,223.915 279.49,224.491 279.587,225.064 
 279.683,225.634 279.779,226.199 279.873,226.76 279.967,227.316 280.059,227.866 280.151,228.411 280.241,228.951 280.329,229.483 280.417,230.01 280.502,230.529 
 280.587,231.041 280.67,231.546 280.751,232.043 280.831,232.532 280.909,233.013 280.985,233.486 281.06,233.951 281.133,234.407 281.204,234.854 281.273,235.292 
 281.34,235.722 281.406,236.142 281.469,236.554 281.531,236.956 281.591,237.348 281.649,237.732 281.705,238.106 281.759,238.471 281.811,238.826 281.861,239.173 
 281.909,239.509 281.955,239.837 281.999,240.155 282.041,240.464 282.081,240.764 282.12,241.055 282.156,241.336 282.19,241.609 282.223,241.873 282.253,242.127 
 282.282,242.373 282.308,242.611 282.333,242.839 282.356,243.059 282.376,243.271 282.395,243.474 282.412,243.669 282.427,243.855 282.441,244.034 282.452,244.204 
 282.461,244.367 282.469,244.521 282.474,2

In [9]:
function trj_generator(sx,xv,tf,te,dt,legnum)#xの遊脚軌道
    #
    #条件ベクトル
    xf=sx[legnum,tf]
    xe=sx[legnum,te]
    vf=xv[tf]*-1
    ve=xv[te]*-1
    A=[xf; xe; vf; ve]
    
    tf=round(tf*dt,6) #時間に戻す
    te=round(te*dt,6) #第6位で切り捨て
        
    B=[tf^3 tf^2 tf 1
        te^3 te^2 te 1
        3tf^2 2tf 1 0
        3te^2 2te 1 0]

    C=inv(B)*A

    #dt=0.01 #離散時間
    t=[i for i in tf:dt:te] #描画時間
    x=C[1]*.^(t,3)+C[2]*.^(t,2)+C[3]*t+C[4] #位置
    v=3*C[1]*.^(t,2)+2*C[2]*t+C[3] #速度
    a=6*C[1]*t+2*C[2]
    
    return x
    
end

function trj_generator_z(zh,zf,ze,ve,tf,te,dt)
    #support time 0.5
    tf=round(tf*dt,7) #時間に戻す
    te=round(te*dt,7) #第6位で切り捨て
    th=round((te-tf)/2+tf,7)
    
    zh=zf+zh
   
    A_z=[zh; ze; zf; ve]

    B_z=[th^3 th^2 th 1
        te^3 te^2 te 1
        tf^3 tf^2 tf 1
        3te^2 2te 1 0]
    C_z=inv(B_z)*A_z
    
    t=[i for i in tf:dt:te]; #描画時間
     z=C_z[1]*.^(t,3)+C_z[2]*.^(t,2)+C_z[3]*t+C_z[4] #位置
    v=3*C_z[1]*.^(t,2)+2*C_z[2]*t+C_z[3] #速度
    a=6*C_z[1]*t+2*C_z[2]
    return z
end



In [10]:
#脚先位置を重心の座標系に変換する
com_trjx=copy(x_plot_log) #重心位置
com_trjy=copy(y_plot_log)

sx_from_com=zeros(3,length(com_trjx))
sy_from_com=zeros(3,length(com_trjy))   #重心からみた支持脚位置
sz_from_com=[-zc for i in 1:length(com_trjx)*3]
sz_from_com=reshape(sz_from_com,3,length(com_trjx))

sx_from_w=reshape(real_sx,3,vpoint_sum)
sy_from_w=reshape(real_sy,3,vpoint_sum)
vz_end=-0.5 #遊脚が接地する速度 -0.1

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum) 
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
        end
    else
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
            
        end
        if(leg_sign[i]!=0)
            #脚を挙げないようにコメントアウト
            sx_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sx_from_com,xv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sy_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sy_from_com,yv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sz_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator_z(0.01,-zc+0.01,-zc,vz_end,1+step,Tsup+step,dt)
        end
        step=step+Tsup
    
        for s in 1:Tdl
            sx_from_com[:,s+step]=sx_from_w[:,i]-com_trjx[s+step]
            sy_from_com[:,s+step]=sy_from_w[:,i]-com_trjy[s+step]
        end
        step=step+Tdl
    end
end
;
#-1が使われている可能性あり

In [11]:
function generate_csv(dataframe)
    CSV.write("leg_trj.csv",dataframe,delim=',',header=false)
end

generate_csv (generic function with 1 method)

In [12]:
df = DataFrame(xr=sx_from_com[1,:],
                xc=sx_from_com[2,:],
                xl=sx_from_com[3,:],
                yr=sy_from_com[1,:],
                yc=sy_from_com[2,:],
                yl=sy_from_com[3,:],
                zr=sz_from_com[1,:],
                zc=sz_from_com[2,:],
                zl=sz_from_com[3,:])
df=generate_csv(df)

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: Base.Dates.DateFormat(Base.Dates.Slot[Base.Dates.DelimitedSlot{Base.Dates.Year}(Base.Dates.Year,'y',4,"-"),Base.Dates.DelimitedSlot{Base.Dates.Month}(Base.Dates.Month,'m',2,"-"),Base.Dates.DelimitedSlot{Base.Dates.Day}(Base.Dates.Day,'d',2,r"(?=\s|$)")],"","english"),IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1),"leg_trj.csv",0,false,String["xr","xc","xl","yr","yc","yl","zr","zc","zl"],false)